In [53]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

import numpy as np
import matplotlib.pyplot as plt 

import time
import os 
import copy 

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cuda


In [55]:
mean = np.array([0.485,0.456,0.406]) #how to get these? similar to fit in keras?
std = np.array([0.299,0.224,0.225])

In [56]:
data_transforms = {'train':transforms.Compose([transforms.RandomResizedCrop(224),
                  transforms.RandomHorizontalFlip(),
                  transforms.ToTensor(),
                  transforms.Normalize(mean,std)]) , 
                  'val': transforms.Compose([transforms.Resize(256),
                  transforms.CenterCrop(224),
                  transforms.ToTensor(),
                  transforms.Normalize(mean,std)]) }

In [57]:
#Dataset

data_dir = "data/hymenoptera_data/"
splits = ['train','val']

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}

In [58]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 244
     Root location: data/hymenoptera_data/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485 0.456 0.406], std=[0.299 0.224 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 153
     Root location: data/hymenoptera_data/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485 0.456 0.406], std=[0.299 0.224 0.225])
            )}

In [59]:
batch_size = 8
dataset_loaders = {x: DataLoader(dataset=image_datasets[x],batch_size=batch_size,shuffle=True,num_workers=0) for x in ['train','val']}


In [60]:
dataset_loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x22c96103b50>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x22c96103a90>}

In [61]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}

In [62]:
dataset_sizes

{'train': 244, 'val': 153}

In [63]:
datasets_classes = image_datasets['train'].classes
num_classes = len(datasets_classes)

print("classes:",datasets_classes)


classes: ['ants', 'bees']


In [77]:
def trainModel(model,loss_func,optimiser,learningrate_scheduler,num_epochs=20):
    start_time = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0 

    steps_per_epoch = {x:len(dataset_loaders['train']) for x in ['train','val']}


    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print("_"*14)

        #Each epoch has training phase and validation phase
        for phase in ['train','val']:

            total_loss = 0
            total_correct_preds = 0


            if phase == 'train':
                model.train() #setting model to train mode

                for i,(inputs,labels) in enumerate(dataset_loaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    #forward pass
                    y_preds = model(inputs)
                    _,pred_labels = torch.max(y_preds,1)

                    loss = loss_func(y_preds,labels)

                    correct_preds = torch.eq(pred_labels,labels).sum().item()

                    total_loss += loss
                    total_correct_preds += correct_preds

                    #backward pass
                    optimiser.zero_grad()
                    loss.backward()

                    #weight updation
                    optimiser.step()
                    
                 
                learningrate_scheduler.step() #changing learning rate after every epoch   
                train_epoch_loss = total_loss/len(dataset_loaders[phase])
                train_epoch_acc = 100.0*total_correct_preds/dataset_sizes[phase]
                print(f"Epoch {epoch}, Training Loss {train_epoch_loss},Training Acc{train_epoch_acc}")    

            else :
                model.eval() #setting model to evaluation mode
                
                with torch.no_grad():
                    for i,(inputs,labels) in enumerate(dataset_loaders[phase]):
                        inputs = inputs.to(device)
                        labels = labels.to(device)

                        #forward pass
                        y_preds = model(inputs)
                        _,pred_labels = torch.max(y_preds,1)

                        loss = loss_func(y_preds,labels)

                        correct_preds = torch.eq(pred_labels,labels).sum().item()

                        total_loss += loss
                        total_correct_preds += correct_preds

                    val_epoch_loss = total_loss/len(dataset_loaders[phase])
                    val_epoch_acc = 100.0*total_correct_preds/dataset_sizes[phase]
                    print(f"Epoch {epoch}, Val Loss {val_epoch_loss},Val Acc{val_epoch_acc}")
                    #Save the best model
                    if val_epoch_acc>best_acc :
                        best_acc = val_epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
                        print("Model at epoch {epoch} saved")
                    
            
            
    print("Loading the best model state")
    model.load_state_dict(best_model_wts)
    return model



In [ ]:
def trainModel1(model,loss_func,optimiser,scheduler,num_epochs):
    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}")
        print("_"*20)

        #training phase
        
        model.train()

        total_loss = 0
        total_correct_preds = 0

        for i,(inputs,labels) in dataset_loaders['train']:
            inputs = inputs.to(device)
            labels = inputs.to(device)

            y_preds = model(inputs)
            _,labels_preds = torch.max(y_preds,1)
            loss = loss_func(y_preds,labels)

            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            total_loss += loss.item()
            total_correct_preds += torch.eq(labels_preds,labels).sum().item()

            
        scheduler.step()

        epoch_loss = total_loss/len(dataset_loaders['train'])
        epoch_acc = 100.0*total_correct_preds/dataset_sizes['train']

        print(f"Train Loss: {epoch_loss}, Acc: {epoch_acc}")

        #validation phase
        model.eval()

        total_loss = 0
        total_correct_preds = 0

        for i,(inputs,labels) in dataset_loaders['val']:
            inputs = inputs.to(device)
            labels = inputs.to(device)

            with torch.no_grad():
                y_preds = model(inputs)
                _,labels_preds = torch.max(y_preds,1)
                loss = loss_func(y_preds,labels)

            

            total_loss += loss.item()
            total_correct_preds += torch.eq(labels_preds,labels).sum().item()


        epoch_loss = total_loss/len(dataset_loaders['val'])
        epoch_acc = 100.0*total_correct_preds/dataset_sizes['val']

        print(f"Val Loss: {epoch_loss}, Acc: {epoch_acc}")


        if epoch_acc>best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    model.load_state_dict(best_model_wts)
    return model


In [65]:
model = models.resnet18(pretrained=True) #loading the pretrained model
#change the last FC layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,num_classes)

model = model.to(device)

In [66]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(),lr=0.001)
#learning schedulers
step_lr_scheduler = lr_scheduler.StepLR(optimiser,step_size=7,gamma=0.1)
#every 7 epochs, learning rate is multiplied by 0.1

In [79]:
trained_model = trainModel(model,loss_func,optimiser, step_lr_scheduler,num_epochs=22)

Epoch 0/21
______________
Epoch 0, Training Loss 0.871651291847229,Training Acc34.83606557377049
Epoch 0, Val Loss 1.0224231481552124,Val Acc29.41176470588235
Model at epoch {epoch} saved
Epoch 1/21
______________
Epoch 1, Training Loss 0.8630949258804321,Training Acc37.295081967213115
Epoch 1, Val Loss 1.071572184562683,Val Acc28.104575163398692
Epoch 2/21
______________
Epoch 2, Training Loss 0.875136137008667,Training Acc34.01639344262295
Epoch 2, Val Loss 1.0724542140960693,Val Acc25.49019607843137
Epoch 3/21
______________


KeyboardInterrupt: 

In [67]:
#freezing convolutional blocks, only train the fully connected layer

model = models.resnet18(pretrained=True) #loading the pretrained model
for param in model.parameters(): #freezing the layers
    param.requires_grad = False
    
#change the last FC layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,num_classes)

model = model.to(device)

In [78]:
trained_model = trainModel(model,optimiser, step_lr_scheduler,loss_func,num_epochs=22)

Epoch 0/21
______________


TypeError: 'SGD' object is not callable